In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
from google.colab import files 

In [ ]:
def CreateModel(archive, training, test, n_classes, name_model):
  !unzip TGM-Images_0.5.zip

  batch_size=256
  image_size=(224, 224)

  train_dataset = image_dataset_from_directory(training,
                                               subset='training',
                                               seed=42,
                                               validation_split=0.1,
                                               batch_size=batch_size,
                                               image_size=image_size)

  validation_dataset = image_dataset_from_directory(training,
                                               subset='validation',
                                               seed=42,
                                               validation_split=0.1,
                                               batch_size=batch_size,
                                               image_size=image_size)

  test_dataset = image_dataset_from_directory(test,
                                               batch_size=batch_size,
                                               image_size=image_size)

  AUTOTUNE = tf.data.experimental.AUTOTUNE

  train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
  validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
  test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

  model = Sequential()
  model.add(Conv2D(16, (5, 5), padding='same', 
                 input_shape=(224, 224, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(128, (5, 5), activation='relu', padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(1024, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(n_classes, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])
  
  history = model.fit(train_dataset, 
                    validation_data=validation_dataset,
                    epochs=5,
                    verbose=2)

  model.save(name_model)

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def EmptyGrid(image, grid):
  img = cv2.imread(image)
  size_x = np.size(img, 1)
  size_y = np.size(img, 0)
  gr_x = size_x
  gr_y = size_y
  img_grd = img
  for a in range(1, grid+1):
    gr_x = round(gr_x/2)
    gr_y = round(gr_y/2)
    for i in range(0, 2**a):
      for j in range(0,2**a):
        x1 = gr_x*i
        x2 = x1+gr_x
        y1 = gr_y*j
        y2 = y1+gr_y
        cv2.rectangle(img_grd, (x1,y1),(x2,y2),(255,255,255), 2)
  img_grd = cv2.cvtColor(img_grd, cv2.COLOR_RGB2BGR)
  plt.imshow(img_grd)

In [ ]:
def SmartGrid(model, labels, image, grid):
  np.set_printoptions(suppress=True)
  model = tf.keras.models.load_model(model, compile=False)
  class_names = open(labels, "r").readlines()
  img_a = cv2.imread(image)
  size_x = np.size(img_a, 1)
  size_y = np.size(img_a, 0)
  gr_x = size_x
  gr_y = size_y
  img_grd = img_a
  for a in range(1, grid+1):
    gr_x = round(gr_x/2)
    gr_y = round(gr_y/2)
    for i in range(0, 2**a):
      for j in range(0,2**a):
        x1 = gr_x*i
        x2 = x1+gr_x
        y1 = gr_y*j
        y2 = y1+gr_y
        cntr = [(x1+x2)/2, (y1+y2)/2]
        img_b = cv2.imread(image)
        img_b = img_b[y1:y2, x1:x2]
        img_b = cv2.resize(img_b, (224, 224), interpolation=cv2.INTER_AREA)
        img_b = np.asarray(img_b, dtype=np.float32).reshape(1, 224, 224, 3)
        img_b = (img_b / 127.5) - 1
        prediction = model.predict(img_b)
        index = np.argmax(prediction)
        class_name = class_names[index].strip()
        confidence_score = prediction[0][index]
        R = int((index % 2) * 255)
        G = int((index % 3) * 127)
        B = int((index % 4) * 84)        
        cv2.rectangle(img_grd, (x1+5,y1+5),(x2-5,y2-5),(R,G,B), 5)
        text_color = (255-R, 255-G, 255-B)
        cv2.putText(img_grd, class_name, (x1+10, y2-10), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
  img_grd = cv2.cvtColor(img_grd, cv2.COLOR_BGR2RGB)
  plt.imshow(img_grd)
  plt.show()

In [ ]:
def SmartList(model, labels, house, image, grid):
  np.set_printoptions(suppress=True)
  model = load_model(model, compile=False)
  class_names = open(labels, "r").readlines()
  img_a = cv2.imread(image)
  size_x = np.size(img_a, 1)
  size_y = np.size(img_a, 0)
  gr_x = size_x
  gr_y = size_y
  img_grd = img_a
  cntrs = []
  for a in range(1, grid+1):
    gr_x = round(gr_x/2)
    gr_y = round(gr_y/2)
    for i in range(0, 2**a):
      for j in range(0,2**a):
        x1 = gr_x*i
        x2 = x1+gr_x
        y1 = gr_y*j
        y2 = y1+gr_y
        xc = round((x1+x2)/2, 0)
        yc = round((y1+y2)/2, 0)
        cntr = [xc, yc]
        img_b = cv2.imread(image)
        img_b = img_b[y1:y2, x1:x2]
        img_b = cv2.resize(img_b, (224, 224), interpolation=cv2.INTER_AREA)
        img_b = np.asarray(img_b, dtype=np.float32).reshape(1, 224, 224, 3)
        img_b = (img_b / 127.5) - 1
        prediction = model.predict(img_b)
        index = np.argmax(prediction)
        confidence_score = prediction[0][index]
        if index == house:
          cv2.rectangle(img_grd, (x1+5,y1+5),(x2-5,y2-5),(100,100,100), 5)
          cntrs.append(cntr)
  return(cntrs)

In [ ]:
!pip install trimesh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.8/680.8 kB 11.5 MB/s eta 0:00:00


In [ ]:
import trimesh
import numpy as np

In [ ]:
def STL_byList(cntrs, x, y, scale, stl):
  
  plane = trimesh.primitives.Box([x, y, 0.01])

  mesh = trimesh.load(stl)
  mesh.apply_scale(scale)

  for i in cntrs:
      x1 = i[0]-x/2
      y1 = i[1]-y/2
      newmesh = mesh.copy()
      newmesh.vertices[:, :2] += np.array([x1, y1])
      plane = trimesh.util.concatenate([plane, newmesh])
  scene = trimesh.Scene(plane)
  return(scene).show()

In [ ]:
def STL_byImage(model, labels, house, image, grid, scale, stl):
  cntrs = SmartList(model, labels, house, image, grid)
  image = cv2.imread(image)
  x = np.size(image, 1)
  y = np.size(image, 0)
  plane = trimesh.primitives.Box([x, y, 0.01])
  mesh = trimesh.load(stl)
  mesh.apply_scale(scale)
  for i in cntrs:
    x1 = i[0]-x/2
    y1 = i[1]-y/2
    newmesh = mesh.copy()
    newmesh.vertices[:, :2] += np.array([x1, y1])
    plane = trimesh.util.concatenate([plane, newmesh])
  scene = trimesh.Scene(plane)
  return(scene).show()